### Question 4.  (10 marks) Find the top-3 states where the ratio of population speaking three languages or more to exactly two languages is the best. Find the worst-3 states as well. The output should contain 6 rows displaying top-3 states (higher to lower ratio) first and then worst-3 states (lower to higher ratio). Call this 3-to-2-ratio.csv and script/program to generate this 3-to-2-ratio.sh. Repeat the question for the ratio of exactly two languages to only one language. Call this 2-to-1-ratio.csv and script/program to generate this 2-to-1-ratio.sh.

# 1. Importing the necessary libraries

In [12]:
import numpy as np
import pandas as pd

# 2. Load census data and clean

In [13]:
# Load the census data
census_data = pd.read_csv('./dataset/DDW_PCA0000_2011_Indiastatedist.csv', dtype='string')
census_data.head()

,State,District,Subdistt,Town/Village,Ward,EB,Level,Name,TRU,No_HH,...,MARG_AL_0_3_F,MARG_HH_0_3_P,MARG_HH_0_3_M,MARG_HH_0_3_F,MARG_OT_0_3_P,MARG_OT_0_3_M,MARG_OT_0_3_F,NON_WORK_P,NON_WORK_M,NON_WORK_F
0,00,000,00000,000000,0000,000000,India,India,Total,249501663,...,5820403,1420348,491424,928924,5518450,3082613,2435837,728966109,291330383,437635726
1,00,000,00000,000000,0000,000000,India,India,Rural,168612897,...,5640822,1154975,386077,768898,3803400,1972348,1831052,485005760,200944045,284061715
2,00,000,00000,000000,0000,000000,India,India,Urban,80888766,...,179581,265373,105347,160026,1715050,1110265,604785,243960349,90386338,153574011
3,01,000,00000,000000,0000,000000,STATE,JAMMU & KASHMIR,Total,2119718,...,44570,26585,6780,19805,108457,56880,51577,8218589,3445572,4773017
4,01,000,00000,000000,0000,000000,STATE,JAMMU & KASHMIR,Rural,1553433,...,40936,21963,5757,16206,89232,45625,43607,5994979,2562471,3432508


## (i) Discard the columns which are not useful for our purpose
- Keep only relevant columns: 'State', 'Level', 'Name', 'TRU', 'TOT_P', 'TOT_M', 'TOT_F'

In [14]:
# Keep relevant columns only
census_data = census_data[['State', 'Level', 'Name', 'TRU', 'TOT_P', 'TOT_M', 'TOT_F']]

## (ii) Convert columns containing numbers to numeric datatype
As of now our dataframe contains all columns read as string datatype, so we will change the datatype of all numeric columns from string to numeric

In [15]:
# Convert the columns containing numbers to numeric datatype
census_data.iloc[:, 4:] = census_data.iloc[:, 4:].apply(pd.to_numeric, errors='ignore')
print('The datatypes of columns containing numeric values has been changed from string to numeric')
print(census_data.dtypes)

The datatypes of columns containing numeric values has been changed from string to numeric
State    string
Level    string
Name     string
TRU      string
TOT_P     int64
TOT_M     int64
TOT_F     int64
dtype: object


## (iii) Save the cleaned dataset for future use

In [16]:
census_data.to_csv('./dataset/census_data_clean.csv', index=False)
census_data.head()

,State,Level,Name,TRU,TOT_P,TOT_M,TOT_F
0,00,India,India,Total,1210854977,623270258,587584719
1,00,India,India,Rural,833748852,427781058,405967794
2,00,India,India,Urban,377106125,195489200,181616925
3,01,STATE,JAMMU & KASHMIR,Total,12541302,6640662,5900640
4,01,STATE,JAMMU & KASHMIR,Rural,9108060,4774477,4333583


# 3. Load Population data and clean
For the purpose of this question we will load the following data file.
- 'C-18 POPULATION BY BILINGUALISM, TRILINGUALISM, AGE AND SEX'

In [17]:
xls_language_data_by_age = pd.read_excel('./dataset/DDW-C18-0000.xlsx', sheet_name='Sheet1', dtype='string')
xls_language_data_by_age.head()

,"C-18 POPULATION BY BILINGUALISM, TRILINGUALISM, AGE AND SEX",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,State,District,Area Name,Total/,Age-group,Number speaking second language,<NA>,<NA>,Number speaking third language,<NA>,<NA>
1,code,code,<NA>,Rural/,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,<NA>,<NA>,<NA>,Urban,<NA>,Persons,Males,Females,Persons,Males,Females
3,1,2,3,4,5,6,7,8,9,10,11
4,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


## (ii) Filter the datasheet and convert to a cleaner dataframe

The excel sheet that we have loaded in the previous cell is difficult to read in pandas. To make our operations easy, we will filter it and give new column names.

The new column names will be as follows:
- 'State_Code',
- 'District_Code',
- 'Area',
- 'TRU',
- 'Age_Group',
- 'Number Speaking Second Language - Persons',
- 'Number Speaking Second Language - Males',
- 'Number Speaking Second Language - Females',
- 'Number Speaking Third Language - Persons',
- 'Number Speaking Third Language - Males',
- 'Number Speaking Third Language - Females'

In [18]:
# Discard the first 4 rows containing column names
language_data_by_age = xls_language_data_by_age.iloc[5:, :].copy()

# Assign the new column names
language_data_by_age.columns = [
    'State_Code',
    'District_Code',
    'Area',
    'TRU',
    'Age_Group',
    'Number Speaking Second Language - Persons',
    'Number Speaking Second Language - Males',
    'Number Speaking Second Language - Females',
    'Number Speaking Third Language - Persons',
    'Number Speaking Third Language - Males',
    'Number Speaking Third Language - Females'
]

language_data_by_age.reset_index(drop=True, inplace=True)  # reset indexes
language_data_by_age.head()

,State_Code,District_Code,Area,TRU,Age_Group,Number Speaking Second Language - Persons,Number Speaking Second Language - Males,Number Speaking Second Language - Females,Number Speaking Third Language - Persons,Number Speaking Third Language - Males,Number Speaking Third Language - Females
0,00,00,INDIA,Total,Total,314988770,176696383,138292387,86009580,50536832,35472748
1,00,00,INDIA,Total,5-9,15649192,8166843,7482349,1844108,978151,865957
2,00,00,INDIA,Total,10-14,34488492,18133423,16355069,7254335,3831131,3423204
3,00,00,INDIA,Total,15-19,42424599,22750908,19673691,12626717,6792766,5833951
4,00,00,INDIA,Total,20-24,41344406,22386694,18957712,12834334,7067614,5766720


## (iii) Convert columns containing numbers to numeric datatype
As of now our dataframe contains all columns read as string datatype, so we will change the datatype of all numeric columns from string to numeric

In [19]:
# Convert the columns containing numbers to numeric datatype
language_data_by_age.iloc[:, 5:] = language_data_by_age.iloc[:, 5:].apply(pd.to_numeric, errors='ignore')
print('The datatypes of columns containing numeric values has been changed from string to numeric')
print(language_data_by_age.dtypes)

The datatypes of columns containing numeric values has been changed from string to numeric
State_Code                                   string
District_Code                                string
Area                                         string
TRU                                          string
Age_Group                                    string
Number Speaking Second Language - Persons     int64
Number Speaking Second Language - Males       int64
Number Speaking Second Language - Females     int64
Number Speaking Third Language - Persons      int64
Number Speaking Third Language - Males        int64
Number Speaking Third Language - Females      int64
dtype: object


# 4. Find the unique state codes in the language data (by age)

In [20]:
# Find the unique state codes in the language data by age
state_codes_from_language_data_by_age = language_data_by_age['State_Code'].dropna().unique()
state_codes_from_language_data_by_age = [state_code for state_code in state_codes_from_language_data_by_age]
print('Number of unique state codes in language data (by age) =', len(state_codes_from_language_data_by_age))

Number of unique state codes in language data (by age) = 36


# 5. Find the top 3 and worst 3 states based on ratio of population speaking exactly two languages to only one language

- Build a dictionary of state_codes as keys and the ratio as values.
- Find the keys corresponding to top 3 and worst three values
- Dump this data to the output csv

In [21]:
# create an empty list that will store [state_code, ratio] as its elements
list_ratio_2_to_1 = []

for state_code in state_codes_from_language_data_by_age:

    # extract the total census data for the current state
    census_data_for_state_total = census_data[(census_data['State'] == state_code)
                                             &
                                             ((census_data['Level'] == 'India') | (census_data['Level'] == 'STATE'))
                                             &
                                             (census_data['TRU'] == 'Total')]

    # extract the total language data for the state
    language_data_for_state_total = language_data_by_age[(language_data_by_age['State_Code'] == state_code)
                                                        &
                                                        (language_data_by_age['TRU'] == 'Total')
                                                        &
                                                        (language_data_by_age['Age_Group'] == 'Total')]
    
    # calculate population speaking exactly two languages = Persons speaking second language - Persons speaking third language
    exactly_two_lang = language_data_for_state_total['Number Speaking Second Language - Persons'].values[0] - \
                       language_data_for_state_total['Number Speaking Third Language - Persons'].values[0]

    # calculate population speaking only one language = Total state population - Persons speaking second language
    only_one_lang = census_data_for_state_total['TOT_P'].values[0] - \
                    language_data_for_state_total['Number Speaking Second Language - Persons'].values[0]

    # calculate the ratio of the above two values and add the data into the list
    list_ratio_2_to_1.append([state_code, exactly_two_lang/only_one_lang])

# 6. Prepare the output file 2-to-1-ratio.csv

The output file will contain 6 rows such that,
- The first 3 rows will contain the top 3 states having highest ratios in higher to lower order
- The next 3 rows will contain the worst 3 states having lowest ratios in lower to higher order.

In [22]:
# Prepare the output file to store the ratio for top 3 and worst 3 states
ratio_2_to_1 = pd.DataFrame(columns=['state-code', '2-to-1-ratio'])

# sort the list based on ratio values, i.e. the second element in each pair in the list
list_ratio_2_to_1.sort(key=lambda x: x[1])

# add the data for top 3 states in higher to lower order
ratio_2_to_1.loc[-1] = list_ratio_2_to_1[-1]
ratio_2_to_1.index += 1
ratio_2_to_1.loc[-1] = list_ratio_2_to_1[-2]
ratio_2_to_1.index += 1
ratio_2_to_1.loc[-1] = list_ratio_2_to_1[-3]
ratio_2_to_1.index += 1

# add the data for worst 3 states in lower to higher
ratio_2_to_1.loc[-1] = list_ratio_2_to_1[0]
ratio_2_to_1.index += 1
ratio_2_to_1.loc[-1] = list_ratio_2_to_1[1]
ratio_2_to_1.index += 1
ratio_2_to_1.loc[-1] = list_ratio_2_to_1[2]
ratio_2_to_1.index += 1

# dump data to csv files
ratio_2_to_1.to_csv('./output/2-to-1-ratio.csv', index=False)
ratio_2_to_1.head()

,state/ut,2-to-1-ratio
5,35,1.499862
4,30,1.157976
3,12,0.938953
2,08,0.105877
1,09,0.114661


--------------------------------------------------------------------------------- END of Q4_2 ---------------------------------------------------------------------------------------------